###Generates Beethoven-style piano music from classical MIDI dataset
Developed a deep learning model that generates original piano music sequences by training an LSTM neural network on classical MIDI files. The model predicts the next note in a sequence and converts the predictions into playable MIDI and WAV files

##Installing required libraries
PyTorch for modeling

music21 for processing MIDI files

midi2audio for converting MIDI → WAV

kaggle API to download the dataset

In [ ]:
!apt-get install -y fluidsynth

In [ ]:
!pip install torch torchvision torchaudio
!pip install music21
!pip install midi2audio
!pip install kaggle

##Downloading Classical Music MIDI Dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!ls -l /root/.kaggle/

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"
!chmod 600 /root/.kaggle/kaggle.json

##Unzip the Classical Music MIDI dataset:

In [ ]:
!kaggle datasets download -d soumikrakshit/classical-music-midi
!unzip -q classical-music-midi.zip -d midi_files

import os
midi_files = [f for f in os.listdir('midi_files') if f.endswith('.mid')]
print(f"Found {len(midi_files)} MIDI files. Sample files: {midi_files[:5]}")


In [ ]:
import os

# Listing all folders/files inside midi_files
for root, dirs, files in os.walk('midi_files'):
    print(f"Folder: {root}, Files: {len(files)}")
    if len(files) > 0:
        print("Sample files:", files[:5])

##Preprocessing MIDI files into notes

In [ ]:
from music21 import converter, instrument, note, chord
import glob

notes = []

# Adjust path to the folder containing MIDI files
midi_files = glob.glob("midi_files/beeth/*.mid")
print(f"Found {len(midi_files)} MIDI files.")

# Parse all MIDI files and extract notes/chords
for file in midi_files:
    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)
    if parts:  # If instruments are separated
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print(f"Total notes extracted: {len(notes)}")
print(f"Unique notes: {len(set(notes))}")

##Create i/o sequences

In [ ]:
import numpy as np

sequence_length = 50

# Map notes to integers
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)
note_to_int = {note: number for number, note in enumerate(pitchnames)}

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

print(f"Number of sequences: {len(network_input)}")

# Reshape for LSTM input
network_input = np.reshape(network_input, (len(network_input), sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = np.array(network_output)

print("network_input shape:", network_input.shape)
print("network_output shape:", network_output.shape)

##LSTM model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Dataset class
class MusicDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.X = torch.tensor(inputs, dtype=torch.float32)
        self.y = torch.tensor(outputs, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Prepare dataset and dataloader
dataset = MusicDataset(network_input, network_output)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# LSTM model
class MusicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(MusicLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MusicLSTM(input_size=1, hidden_size=256, output_size=n_vocab).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

##Training

In [ ]:
epochs = 50
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

##Generate music sequences

In [ ]:
from music21 import stream, note, chord
import numpy as np
import torch

model.eval()  # switch to evaluation mode

# Pick a random seed sequence from the input
start_idx = np.random.randint(0, len(network_input)-1)
pattern = network_input[start_idx].tolist()

output_notes = []

# Generate 500 notes
for note_index in range(1000):
    prediction_input = torch.tensor([pattern], dtype=torch.float32).to(device)
    prediction = model(prediction_input)
    import torch.nn.functional as F

    temperature = 1.0  # adjust creativity (0.7(repetitive) or 1.0 (safe) or 1.2(risky))
    prediction_probs = F.softmax(prediction / temperature, dim=1)
    index = torch.multinomial(prediction_probs, num_samples=1).item()
    result = pitchnames[index]

    output_notes.append(result)

    # Append new note and remove first to maintain sequence length
    pattern.append([index/float(n_vocab)])
    pattern = pattern[1:]

print(f"Generated {len(output_notes)} notes.")

##Converting generated notes to MIDI and play

In [ ]:
from music21 import stream, note, chord
from midi2audio import FluidSynth
from IPython.display import Audio

# Convert notes/chords to music21 stream
offset = 0
output_stream = stream.Stream()

for pattern_note in output_notes:
    # Handle chords
    if '.' in pattern_note or pattern_note.isdigit():
        notes_in_chord = pattern_note.split('.')
        chord_notes = []
        for n in notes_in_chord:
            new_note = note.Note(int(n))
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = offset
        output_stream.append(new_chord)
    else:
        new_note = note.Note(pattern_note)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_stream.append(new_note)
    offset += 0.5

# Save as MIDI
output_stream.write('midi', fp='generated_music.mid')

# Convert to WAV and play
fs = FluidSynth()
fs.midi_to_audio('generated_music.mid', 'generated_music.wav')

Audio('generated_music.wav')